In [ ]:
# imports and setup
import re
import ast
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_csv(r"E:\df_final.csv")

In [ ]:
# Data cleaning

df_copy = df.copy()

# Drop duplicates
df_copy = df_copy.drop_duplicates(subset=['app_profile_url'])

# General
cols_to_keep = ['app_id','name','categories','category_chart_type_pairs','contains_ads','file_size','has_iap','installs','minimum_os_version','most_popular_country','price','publisher_country','publisher_name','rating','rating_count','rating_breakdown','release_date','top_in_app_purchases','valid_countries']
for c in df_copy.columns:
    if c not in cols_to_keep:
        df_copy = df_copy.drop(c, axis = 1)

# Categories
genre_list = []
for category in df_copy['categories']:
    genre_list.append(ast.literal_eval(category)[0]['name'])
df_copy['categories'] = genre_list

# Top Charts
chart_list = []
for chart in df_copy['category_chart_type_pairs']:
    if ast.literal_eval(chart):
        chart_list.append(1)
    else:
        chart_list.append(0)
df_copy['top_charts'] = chart_list
df_copy = df_copy.drop('category_chart_type_pairs',axis=1)

# Contains ads
df_copy["contains_ads"] = df_copy["contains_ads"].fillna(0).astype(int)

# Has iap
df_copy['has_iap'] = df_copy['has_iap'].fillna(0).astype(int)

# File Size
fixed_size = []
for size in df_copy['file_size']:
    if type(size) is str:
        if size[len(size) - 1] == 'M':
            fixed_size.append(float(size.strip('M').replace(',', '')))
        elif size[len(size) - 1] == 'G':
            fixed_size.append(float(size.strip('G').replace(',', '')) * 1000)
        else:
            fixed_size.append(0)
    else:
        # 0 for "Varies with device"
        fixed_size.append(0)
df_copy['file_size'] = fixed_size

# Installs
df_copy = df_copy.dropna(subset = ['installs'])
df_copy['installs'] = [int(value.strip('+').replace(',', '')) for value in df_copy['installs']]

# In-App Purchases
fixed_iap_from = []
fixed_iap_up_to = []
for purch in df_copy['top_in_app_purchases']:
    fixed_purch = purch.strip(' per item')
    splited_purch = fixed_purch.split(' - ')
    if splited_purch[0] != fixed_purch:
        fixed_iap_from.append(float(splited_purch[0].strip('$')))
        fixed_iap_up_to.append(float(splited_purch[1].strip('$')))
    elif purch == '[]':
        fixed_iap_from.append(0)
        fixed_iap_up_to.append(0)
    else:
        fixed_iap_from.append(float(fixed_purch.strip('$')))
        fixed_iap_up_to.append(float(fixed_purch.strip('$')))
df_copy['in_app_purchases_from'] = fixed_iap_from
df_copy['in_app_purchases_up_to'] = fixed_iap_up_to
df_copy = df_copy.drop('top_in_app_purchases', axis = 1)

# Minimum Os Version
fixed_minimum_version = []
for ver in df_copy['minimum_os_version']:
    if type(ver) is str:
        # Checks if the first letter is a digit
        if re.search('\d',ver):
            fixed_minimum_version.append(int(re.findall(r'\d\d?', ver)[0]))
        else:
            fixed_minimum_version.append(0)
    else:
        fixed_minimum_version.append(0)
df_copy['minimum_os_version'] = fixed_minimum_version

# Price
df_copy['price'] = np.round(df_copy['price'], 2)

# Rating Breakdown
stars = [[] for i in range(5)]
for rating_breakdown in df_copy['rating_breakdown']:
    for idx, star_count in enumerate(ast.literal_eval(rating_breakdown)):
       stars[idx].append(star_count)
for num, lst_stars in enumerate(stars):
    df_copy[f'stars_{num + 1}'] = lst_stars
df_copy = df_copy.drop('rating_breakdown',axis=1)

# Release Date
date_lst = []
for date in df_copy['release_date']:
    if type(date) is str:
        date_lst.append(datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ').year)
    else:
        date_lst.append(0)
df_copy['release_date'] = date_lst

# Rearrange the dataframe
df_copy = df_copy[['app_id', 'name', 'categories', 'price', 'installs', 'rating', 'rating_count', 'stars_5', 'stars_4', 'stars_3', 'stars_2', 'stars_1', 'has_iap', 'in_app_purchases_from', 'in_app_purchases_up_to', 'contains_ads', 'top_charts', 'most_popular_country', 'valid_countries', 'publisher_name', 'publisher_country', 'release_date', 'file_size', 'minimum_os_version']]

# Categories columns
# TODO : might need to add more cols to the list.
cols = ['categories', 'has_iap', 'contains_ads', 'top_charts', 'most_popular_country', 'publisher_country', 'release_date', 'minimum_os_version', 'publisher_name']
df_copy[cols] = df_copy[cols].astype('category')

# Outliers
'''Handles outliers in price column'''
z_score = (df_copy['price'] - df_copy['price'].mean()) / df_copy['price'].std()
outliers = abs(z_score) > 7
df_copy['price'][outliers] = np.nan
df_copy = df_copy[df_copy['price'].notna()]
#df_copy.dropna(subset=['price'])

# Print head
df_copy.head()

In [ ]:
df_copy.to_csv(r'df_games(ver1.1).csv',encoding='utf-8-sig')